# Extraindo Dados de Qualquer Site

### Helder C. Silva

## Introdução

Nesse experimento o objetivo é demonstrar como extrair dados de qualquer site aplicando o método chamado **Web Scraping**.

O **web Scraping** (raspagem de rede), também conhecido como extração de dados da web, é o nome dado ao processo de coleta de dados estruturados da web de maneira automatizada. Via de regra, essa metodologia é uasada por pessoas e empresas que desejam usar uma vasta quantidade de dados da web disponíveis publicamente para tomar decisões mais inteligente. 

O processo de raspagem básico se consiste no ato de copiar e colar informações de um site para outro meio, a diferença é que a raspagem de rede faz isso em uma escala minúscula e com automação inteligente, para extrair milhões de dados de páginas da internet. 

Lembrando que é um processo legal, tendo em vista que os dados estão livremente disponíveis, tendo em vista que só vamos extrair os dados que estão ali na tela de forma automatizada.



## Sobre a fonte dos dados

Vamos extrair os dados do site: https://www.fundamentus.com.br/ que é um sitema on-line que disponibiliza informações financeiras e fundamentalistas das empresas com ações listadas na Bovespa. Tendo em vista que o mesmo apresenta um completo banco de dados apresentado de forma acessível para auxiliar o investidor a encontrar as melhores opções de investimento. 

In [1]:
#instalando a biblioteca

!pip install bs4

In [2]:
#importando bibliotecas
from urllib.request import urlopen, urlretrieve, Request
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup
import pandas as pd
 
#Obtendo o HTML
url = "https://www.fundamentus.com.br/resultado.php"
headers ={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36"}
 
req = Request(url,headers=headers) 
response = urlopen(req) #abrir a url
html = response.read() #fazer a leitura do response
soup = BeautifulSoup(html,'html.parser') 


#Obtendo as TAGs de interesse ( topo das informações)
lista =soup.find('table') #procurar uma tabela e trazer os resultados pra dentro da lista


In [3]:
#quantidade ações
qtd = soup.findAll('span',class_='tips') 
qtd = range(int(len(qtd)-1)) #retira o ultimo registro para não ter erro
 
#Declarando variáveis cards
resumo = []

In [4]:
#pega as primeiras informações que não entram no for
papel =  lista.find('td').find('span',class_='tips').getText()
cotacao = lista.find('td').findNext('td').contents[0]
 
for i in qtd: #looping que vai fazer a verificação passando por todas as colunas
 
  acoes ={}
    
  PL = cotacao.findNext('td').contents[0]
  PVP = PL.findNext('td').contents[0]
  PSR = PVP.findNext('td').contents[0]
  DividendYied = PSR.findNext('td').contents[0]
  PAtivo = DividendYied.findNext('td').contents[0]
  PCapGiro = PAtivo.findNext('td').contents[0]
  PEbit= PCapGiro.findNext('td').contents[0]
  PAtivoCirc= PEbit.findNext('td').contents[0]
  EVEbit= PAtivoCirc.findNext('td').contents[0]
  EVEbita= EVEbit.findNext('td').contents[0]
  MrgEbit= EVEbita.findNext('td').contents[0]
  MrgLiq= MrgEbit.findNext('td').contents[0]
  LiqCorrente= MrgLiq.findNext('td').contents[0]
  ROIC= LiqCorrente.findNext('td').contents[0]
  ROE= ROIC.findNext('td').contents[0]
  Liq2Meses= ROE.findNext('td').contents[0]
  PatriLiquido= Liq2Meses.findNext('td').contents[0]
  DivBruta_por_Patri= PatriLiquido.findNext('td').contents[0]
  Cresc_5a= DivBruta_por_Patri.findNext('td').contents[0]
 
  acoes['id']= i
  acoes['Papel'] = papel
  acoes['Cotacao'] = cotacao
  acoes['PL'] = PL
  acoes['PVP']=PVP
  acoes['DividendYied']=DividendYied
  acoes['PAtivo']=PAtivo
  acoes['PCapGiro']=PCapGiro
  acoes['PEbit']=PEbit
  acoes['PAtivoCirc']=PAtivoCirc
  acoes['EVEbit']=EVEbit
  acoes['EVEbita']=EVEbita
  acoes['MrgEbit']=MrgEbit
  acoes['MrgLiq']=MrgLiq
  acoes['LiqCorrente']=LiqCorrente
  acoes['ROIC']=ROIC
  acoes['ROE']=ROE
  acoes['Liq2Meses']=Liq2Meses
  acoes['PatriLiquido']=PatriLiquido
  acoes['DivBruta_por_Patri']=DivBruta_por_Patri
  acoes['Cresc_5a']=Cresc_5a
 
  #Adiciona o dicionário de ações em uma lista
  resumo.append(acoes) #acabou uma linha adiciona no resumo
 
  #try retorna erro por a ultima linha não encontra o span
  try:
    papel = Cresc_5a.findNext('td').span.a.contents[0]
    cotacao = papel.findPrevious('td').findNext('td').contents[0]
 
  except HTTPError as e:
    print(e.status, e.reason)

In [5]:
#cria data frame
dataset = pd.DataFrame(resumo) #armazenar diversos campos
 
dataset.head(10000)

,id,Papel,Cotacao,PL,PVP,DividendYied,PAtivo,PCapGiro,PEbit,PAtivoCirc,...,EVEbita,MrgEbit,MrgLiq,LiqCorrente,ROIC,ROE,Liq2Meses,PatriLiquido,DivBruta_por_Patri,Cresc_5a
0,0,PORP4,"2,40","0,00","0,00","0,00%","0,000","0,00","0,00","0,00",...,"0,00","0,00%","0,00%","0,00","0,00%","-2,08%","0,00","22.399.000,00","0,00","13,66%"
1,1,PMET3,"0,00","0,00","0,00","0,00%","0,000","0,00","0,00","0,00",...,"0,00","0,00%","0,00%","0,00","0,00%","4,10%","0,00","-290.863.000,00","0,00","37,74%"
2,2,MNSA4,"0,47","0,00","0,00","0,00%","0,000","0,00","0,00","0,00",...,"0,00","-208,15%","-362,66%","3,63","-13,50%","145,70%","0,00","-9.105.000,00","-6,52","-41,11%"
3,3,CSTB4,"147,69","0,00","0,00","0,00%","0,000","0,00","0,00","0,00",...,"0,00","40,85%","28,98%","2,60","22,40%","20,11%","0,00","8.420.670.000,00","0,14","31,91%"
4,4,CFLU4,"1.000,00","0,00","0,00","0,00%","0,000","0,00","0,00","0,00",...,"0,00","8,88%","10,72%","1,10","17,68%","32,15%","0,00","60.351.000,00","0,06","8,14%"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981,981,LWSA3,"10,89","943,61","2,18","0,00%","1,411","5,69","343,07","12,99",...,"43,96","1,91%","0,69%","2,16","0,61%","0,23%","91.481.200,00","2.959.030.000,00","0,03","51,64%"
982,982,UBBR11,"14,75","1.201,81","3,91","0,00%","0,000","0,00","0,00","0,00",...,"0,00","0,00%","0,00%","0,00","0,00%","0,33%","0,00","10.317.200.000,00","0,00","10,58%"
983,983,UBBR3,"18,00","1.466,61","4,77","0,00%","0,000","0,00","0,00","0,00",...,"0,00","0,00%","0,00%","0,00","0,00%","0,33%","0,00","10.317.200.000,00","0,00","10,58%"
984,984,CALI4,"50,00","4.770,51","39,82","0,86%","0,803","1,62","51,65","-2,75",...,"76,25","4,83%","0,05%","3,58","1,79%","0,83%","0,00","4.672.000,00","26,54","107,87%"


## Tratamento dos Dados

In [6]:
#Todos os elementos que tem vírgula (,), vamos alterar para ponto(.), pois são valores. 
#conversão em float

#TRATAMENTO DE DADOS CAMPO PL  
dataset['PL'] = dataset['PL'].str.replace('.', '', regex=True).replace(',', '.', regex=True)
convert_dict = {'PL': float}
dataset['PL']  = dataset['PL'].astype(convert_dict)
 
#TRATAMENTO DE DADOS CAMPO ROE
dataset['ROE'] = dataset['ROE'].str.replace('.', '', regex=True).replace(',', '.', regex=True).replace('%', '', regex=True)
convert_dict = {'ROE': float}
dataset['ROE']  = dataset['ROE'].astype(convert_dict)/100
 
#TRATAMENTO DE DADOS CAMPO MrgLiq
dataset['MrgLiq'] = dataset['MrgLiq'].str.replace('.', '', regex=True).replace(',', '.', regex=True).replace('%', '', regex=True)
convert_dict = {'MrgLiq': float}
dataset['MrgLiq']  = dataset['MrgLiq'].astype(convert_dict)/100
 
#TRATAMENTO DE DADOS CAMPO DIVIDA BRUTA/PATRIMONIO  
dataset['DivBruta_por_Patri'] = dataset['DivBruta_por_Patri'].str.replace('.', '', regex=True).replace(',', '.', regex=True)
convert_dict = {'DivBruta_por_Patri': float}
dataset['DivBruta_por_Patri']  = dataset['DivBruta_por_Patri'].astype(convert_dict)
 
#TRATAMENTO DE DADOS CAMPO CAGER
dataset['Cresc_5a'] = dataset['Cresc_5a'].str.replace('.', '', regex=True).replace(',', '.', regex=True).replace('%', '', regex=True)
convert_dict = {'Cresc_5a': float}
dataset['Cresc_5a']  = dataset['Cresc_5a'].astype(convert_dict)/100
 
#TRATAMENTO DE DADOS DIVIDEND YIED
dataset['DividendYied'] = dataset['DividendYied'].str.replace('.', '', regex=True).replace(',', '.', regex=True).replace('%', '', regex=True)
convert_dict = {'DividendYied': float}
dataset['DividendYied']  = dataset['DividendYied'].astype(convert_dict)/100
 
#TRATAMENTO DE DADOS CAMPO ROIC
dataset['ROIC'] = dataset['ROIC'].str.replace('.', '', regex=True).replace(',', '.', regex=True).replace('%', '', regex=True)
convert_dict = {'ROIC': float}
dataset['ROIC']  = dataset['ROIC'].astype(convert_dict)/100
 
#TRATAMENTO DE DADOS CAMPO ROIC
dataset['MrgEbit'] = dataset['MrgEbit'].str.replace('.', '', regex=True).replace(',', '.', regex=True).replace('%', '', regex=True)
convert_dict = {'MrgEbit': float}
dataset['MrgEbit']  = dataset['MrgEbit'].astype(convert_dict)/100
 
#TRATAMENTO DE DADOS CAMPO PVP  
dataset['PVP'] = dataset['PVP'].str.replace('.', '', regex=True).replace(',', '.', regex=True)
convert_dict = {'PVP': float}
dataset['PVP']  = dataset['PVP'].astype(convert_dict)
 
 
#TRATAMENTO DE DADOS CAMPO EVEbit
dataset['EVEbit'] = dataset['EVEbit'].str.replace('.', '', regex=True).replace(',', '.', regex=True)
convert_dict = {'EVEbit': float}
dataset['EVEbit']  = dataset['EVEbit'].astype(convert_dict)
 
#TRATAMENTO DE DADOS CAMPO EVEbita
dataset['EVEbita'] = dataset['EVEbita'].str.replace('.', '', regex=True).replace(',', '.', regex=True)
convert_dict = {'EVEbita': float}
dataset['EVEbita']  = dataset['EVEbita'].astype(convert_dict)

In [7]:
#Blacklist de ações descontinuadas
blacklist ={
    'PTPA3'
}
 
#Filtragem de dados
selecao = (dataset['PL'] >= 1) & (dataset['ROE'] > 0) & (dataset['ROE'] < 90) &  (dataset['MrgLiq'] > 0) & (dataset['DivBruta_por_Patri'] > 1.3) &  (dataset['Cresc_5a'] > 0.1)
 
melhores_acoes = dataset[selecao].sort_values('PL', ascending=False)
melhores_acoes

,id,Papel,Cotacao,PL,PVP,DividendYied,PAtivo,PCapGiro,PEbit,PAtivoCirc,...,EVEbita,MrgEbit,MrgLiq,LiqCorrente,ROIC,ROE,Liq2Meses,PatriLiquido,DivBruta_por_Patri,Cresc_5a
984,984,CALI4,"50,00",4770.51,39.82,0.0086,"0,803","1,62","51,65","-2,75",...,76.25,0.0483,0.0005,"3,58",0.0179,0.0083,"0,00","4.672.000,00",26.54,1.0787
980,980,CALI3,"8,21",783.32,6.54,0.0526,"0,132","0,27","8,48","-0,45",...,36.40,0.0483,0.0005,"3,58",0.0179,0.0083,"4.021,27","4.672.000,00",26.54,1.0787
959,959,CEPE3,"128,00",115.06,6.73,0.0000,"0,711","5,91","9,35","-1,19",...,11.52,0.1278,0.0104,"1,68",0.0894,0.0585,"0,00","1.420.000.000,00",5.40,0.1456
935,935,RDOR3,"33,28",50.44,4.94,0.0126,"1,193","4,20","17,31","-3,95",...,15.73,0.1788,0.0639,"2,87",0.0946,0.0979,"168.898.000,00","13.542.900.000,00",2.19,0.2931
921,921,CEPE6,"43,00",38.65,2.26,0.1094,"0,239","1,98","3,14","-0,40",...,6.94,0.1278,0.0104,"1,68",0.0894,0.0585,"399,44","1.420.000.000,00",5.40,0.1456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,419,PTBL4,"2,59",1.61,0.81,0.0000,"0,164","1,53","0,94","-0,49",...,1.92,0.1830,0.1064,"1,30",0.2448,0.5009,"0,00","453.307.000,00",1.61,0.2099
417,417,SUZB5,"21,13",1.58,1.03,0.0000,"0,224","1,23","1,50","-0,45",...,3.28,0.4077,0.3889,"2,81",0.1814,0.6524,"0,00","27.951.700.000,00",2.72,0.3551
412,412,SUZB6,"17,88",1.33,0.87,0.0000,"0,189","1,04","1,27","-0,38",...,3.12,0.4077,0.3889,"2,81",0.1814,0.6524,"0,00","27.951.700.000,00",2.72,0.3551
411,411,EBEN4,"17,51",1.27,0.75,0.0000,"0,106","2,15","0,67","-0,20",...,2.60,0.1733,0.0909,"1,18",0.1964,0.5882,"0,00","914.931.000,00",3.34,0.1036


## Conclusão

Durante o experimento foi possível realizar a extração dos dados através de um site no caso em questão foi o **Fundamentus**, contudo poderia ser qualquer site.

A técnica utilizada (web scraping) possibilita a aquisição de grandes quantidades de dados em tempo reduzido, permitindo assim análises e estudos para desenvolvimento inclusive de modelos de inteligêncial artificial. 